In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-08-10 08:03:39.146799: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 08:03:39.207889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 08:03:39.208915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 08:03:40.158630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
filename='intents_labeled_by_9'

train_file = f'/home/azureuser/projects/aerius/ai/myapp/chatbot/datas/{filename}.csv'
data = pd.read_csv(train_file, delimiter = ',')
queries = data['query'].tolist()
intents = data['label'].tolist()

print(data.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                              query    intent  label
0  아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?  주문_취소_확인      7
1             실수로 취소하면 재주문해야 하는 거죠?  주문_취소_확인      7
2                     택배비 따로 추가되나요?  배송_비용_질문      5
3                          택배비 있나요?  배송_비용_질문      5
4                        택배비 따로 들어요  배송_비용_질문      5
len of queries =  235867
len of intents =  235867


In [3]:
encoder = LabelEncoder()
encoder.fit(intents)

encoded_intents = encoder.transform(intents)
num_classes = len(encoder.classes_)
print(f"The number of unique labels is {num_classes}")

The number of unique labels is 9


In [4]:
from ai.myapp.chatbot.utils.Preprocess import Preprocess
p = Preprocess()

words = []
for sentence in queries:
    if isinstance(sentence, str):  # Only process if the sentence is a string
        preprocessed = p.delete_intent_trash_tags(sentence=sentence)
        word_list, _ = p.divide_words_tags(preprocessed)
        words.extend(word_list)
    else:
        print(f"Found non-string value: {sentence}")


# Initialize the tokenizer
p.initialize_tokenizer(words)

# Convert the queries into sequences
sequences = []
for sentence in queries:
    sequence = p.text_to_sequence(sentence)
    sequences.append(sequence)

None


In [6]:
from ai.backend.settings import INTENT_MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=INTENT_MAX_SEQ_LEN, padding='post')

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, encoded_intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

vocab_size = len(p.tokenizer.word_index) + 1  # 단어 사전의 크기
embedding_dim = 30  # 임베딩 벡터의 차원
lstm_units = 200  # LSTM 레이어의 유닛 수
output_classes = len(set(encoded_intents))  # 고유한 인텐트 레이블의 수
max_len = INTENT_MAX_SEQ_LEN

# 모델 구축
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, dropout=0.50, recurrent_dropout=0.25)),
    tf.keras.layers.Dense(output_classes, activation='softmax')
])

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])

# one-hot 인코딩된 레이블을 사용하여 학습
y_train_onehot = tf.keras.utils.to_categorical(np.concatenate([y for _, y in train_ds], axis=0), num_classes=output_classes)
y_val_onehot = tf.keras.utils.to_categorical(np.concatenate([y for _, y in val_ds], axis=0), num_classes=output_classes)

train_ds_onehot = tf.data.Dataset.from_tensor_slices((np.concatenate([x for x, _ in train_ds], axis=0), y_train_onehot)).batch(20)
val_ds_onehot = tf.data.Dataset.from_tensor_slices((np.concatenate([x for x, _ in val_ds], axis=0), y_val_onehot)).batch(20)

history = model.fit(train_ds_onehot, validation_data=val_ds_onehot, epochs=10)
loss, accuracy = model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(accuracy * 100))


Epoch 1/10


ValueError: in user code:

    File "/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/home/azureuser/projects/aerius/ven

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. 모델 예측
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# test_ds_resampled에서 라벨만 추출
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# 2. 성능 지표 계산
precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

1180/1180 [==============================] - 17s 10ms/step
Precision: 0.1633
Recall: 0.1834
F1 Score: 0.1655
Accuracy: 0.1834


In [ ]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 1.2062445878982544
Test accuracy: 0.5604596138000488


In [ ]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'Accuracy: {accuracy * 100}')
print(f'loss: {loss}')

model.save(f'/home/azureuser/projects/aerius/ai/myapp/chatbot/models/{filename}.h5')

1180/1180 [==============================] - 13s 10ms/step - loss: 1.2050 - accuracy: 0.5607
Accuracy: 56.06715679168701
loss: 1.2050210237503052


/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
